# NewsAPI Classification and Text Analysis
Authors: Christian Lee, Anahit Shekikyan, Graham Ward 

## Library Importing

In [ ]:
import nltk
import random
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import html

from wordcloud import WordCloud
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation

nltk.download('stopwords')

## Function Creations

Please use this section to develop functions or place them here from previous assignments

In [ ]:
sw = stopwords.words("english")

punctuation = set(punctuation) # speeds up comparison

# create compiled regex obj matching one or more whitespace
ws_pattern = re.compile(r"\s+")

In [ ]:
# Function creations

# Functions from previous assignments before cell
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity (https://en.wikipedia.org/wiki/Lexical_diversity), 
        and num_tokens most common tokens. Return a list with the number of tokens, number
        of unique tokens, lexical diversity, and number of characters. 
    
    """

    # Fill in the correct values here. 
    total_tokens = len(tokens)
    num_unique_tokens = len(set(tokens))
    lexical_diversity = num_unique_tokens / total_tokens if total_tokens > 0 else 0
    num_characters = sum(len(token) for token in tokens)
    
    if verbose :        
        print(f"There are {total_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        token_counts = Counter(tokens)
        print(f"The {num_tokens} most common tokens are:")
        for token, count in token_counts.most_common(num_tokens):
            print(f" {token}: {count}")
                    
    return([total_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])

# construct function for cleaning and tokenizing
def clean_and_tokenize(text, stopwords):
    """
    Clean and tokenize a single speech from the conventions.

    This function applies the following steps:
      1. Converts all characters to lowercase.
      2. Removes punctuation characters by replacing them with spaces.
      3. Collapses multiple whitespace characters into a single space and strips leading/trailing spaces.
      4. Splits the cleaned string into tokens on whitespace.
      5. Removes any tokens that are in the provided stopword set.

    Parameters
    ----------
    text : str
        A single text of a speech to clean and tokenize.
    stopwords : set
        A set of stopwords (all lowercase) to filter out from the resulting tokens.

    Returns
    -------
    list of str
        A list of cleaned tokens with punctuation removed, normalized case, and stopwords excluded.
    """
    # handle coercion to string format error handling
    if text is None:
        return []
    if not isinstance(text, str):
        text = str(text)
    # make lowercase
    text = text.lower()
    # punctuation removal
    text = "".join(ch if ch not in punctuation else " " for ch in text)
    # include the removal of special characters
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    # replace whitespace with single whitespace and remove leading/trailing ws
    text = ws_pattern.sub(" ", text).strip()
    # split cleaned string on space
    tokens = text.split()
    # remove stopwords
    return [t for t in tokens if t not in stopwords]

# special character locator and counter
def find_special_characters(df, columns):
    """
    Identify all unique special characters in one or more DataFrame columns,
    and return their counts.
    
    Args:
        df (pd.DataFrame): Input dataframe
        columns (str or list): Column name(s) to search
    
    Returns:
        dict: Dictionary of special characters and their counts
    """
    if isinstance(columns, str):  # allow a single column
        columns = [columns]
        
    pattern = re.compile(r"[^a-zA-Z0-9\s]")
    counter = Counter()
    
    for col in columns:
        for text in df[col].dropna():
            matches = pattern.findall(str(text))
            counter.update(matches)
    
    return dict(counter)


## Data Importing

In [ ]:
# import the articles csv into a pandas dataframe for manipulation
df = pd.read_csv('articles.csv')

# print the head of the dataframe
print(df.head())

## Initial Descriptive Stats and Data Exploration

In [ ]:
# print out the unique colummns of the dataframe
print(df.columns)
# print the data types of each column
print("\n", df.dtypes)

In [ ]:
# print the number of rows in the dataframe
print("The number of rows in the dataframe are: ", df.shape[0])

In [ ]:
# get counts for each category
article_counts = df['category'].value_counts().reset_index()
# counts
article_counts.columns = ['category', 'count']
# print
print(article_counts)

In [ ]:
# visualization
ax = sns.barplot(x='category', y='count', data=article_counts)

ax.bar_label(ax.containers[0])
plt.xticks(rotation=90)
plt.title("Category Counts")
plt.show()

In [ ]:
# Find and get special character counts for contents
content_special_characters = find_special_characters(df, ["content", "description", "title"])

print(content_special_characters)

In [ ]:
# find and get special characters counts for description
desc_special_characters = find_special_characters(df, ["content", "description", "title"])

print(desc_special_characters)

In [ ]:
# Find and get special character counts for titles
title_special_characters = find_special_characters(df, ["content", "description", "title"])

print(title_special_characters)

In [ ]:
# identify any missing values in each column
print("Missing values for each column")
print(df.isnull().sum())

# percentage
print("\nPercentage of missing per column: ")
print((df.isnull().mean() * 100).round(2))

# number of rows missing both description and content
both_missing = df[df['description'].isnull() & df['content'].isnull()]
print(f"\nThe number of observations missing BOTH description and content is: {len(both_missing)}")

## Data Cleaning and Preprocessing

In [ ]:
# establish a new df_clean object for cleaning to keep original df
df_clean = df.copy()

# Establish regex patterns for removal and precleaning
# html pattern
html_tag_regex = re.compile(r"<[^>]+>")
# removal of number of characters left in content
truncate_tail_regex = re.compile(r"\[\s*\+\s*\d+\s*chars?\s*\]")
# trailing ellipses at ends 
ellipsis_end_regex = re.compile(r"(…|\.{3,})\s*$")

# text precleaning to remove 
def preclean_text(s: str) -> str:
    """
    Pre-clean raw text from the NewsAPI dataset by stripping noise and formatting artifacts.

    Steps performed:
        1. Decode HTML entities (e.g., "&amp;" → "&").
        2. Remove inline HTML tags (e.g., "<p>", "<a href=...>").
        3. Strip NewsAPI truncation markers of the form "[+123 chars]".
        4. Trim trailing ellipses at the end of text ("...", "……").
        5. Collapse multiple whitespace characters into a single space and strip edges.

    Parameters
    ----------
    s : str
        The input text string to be cleaned. Non-string or empty inputs are returned unchanged.

    Returns
    -------
    str
        A cleaned text string suitable for further cleaning.
    """
    # handles null values
    if not isinstance(s, str) or not s.strip():
        return s
    # convert html entries to normal text
    x = html.unescape(s)
    # remove HTML tags
    x = html_tag_regex.sub(" ", x)
    # remove truncation tokens
    x = truncate_tail_regex.sub(" ", x)
    # trim trailing ellipses
    x = ellipsis_end_regex.sub(" ", x)
    # collapse whitespace and reduce leading and trailing ws
    x = ws_pattern.sub(" ", x).strip()
    return x

# apply precleaning function to the text columns
for col in ['content', 'description', 'title']:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].apply(preclean_text)

In [ ]:
# apply the cleaning and tokenization to each column with text data.
df_clean['title_tokens'] = df_clean['title'].apply(lambda x: clean_and_tokenize(x, sw))
df_clean['desc_tokens'] = df_clean['description'].apply(lambda x: clean_and_tokenize(x, sw))
df_clean['content_tokens'] = df_clean['content'].apply(lambda x: clean_and_tokenize(x, sw))

# get some token counts
df_clean['num_title_tokens'] = df_clean['title_tokens'].apply(len)
df_clean['num_desc_tokens'] = df_clean['desc_tokens'].apply(len)
df_clean['num_content_tokens'] = df_clean['content_tokens'].apply(len)

# view head of clean data
print(df_clean.head())

We still need to determine how we are going to handle the missing values for the description and content columns. Easiest recommendation is to just drop, we lose 10.5% of the content data, but can always rerun the older_fetch_articles python script to grab more data if needed as long as we build out this notebook.

## Exploratory Data Analysis

In [ ]:
# get the new shape of the dataframe after cleaning
print(f"The new dimensions of the data are : {df_clean.shape}")

In [ ]:
# identify any missing values in each column
print("Missing values for each column")
print(df_clean.isnull().sum())

# percentage
print("\nPercentage of missing per column: ")
print((df_clean.isnull().mean() * 100).round(2))

# number of rows missing both description and content
both_missing = df_clean[df_clean['description'].isnull() & df_clean['content'].isnull()]
print(f"\nThe number of observations missing BOTH description and content is: {len(both_missing)}")

## Feature Engineering

In [ ]:
df_clean['comb_text'] = df_clean['title'] + ' ' + df_clean['description'] + ' ' + df_clean['content']
df_clean['comb_tokens'] = df_clean['comb_text'].apply(lambda x: clean_and_tokenize(x, sw))

In [ ]:
# Build out wordclouds for the content_tokens of each category.
# Config
MAX_WORDS = 200
MIN_TOKEN_LEN = 2
# pixel size passed to WordCloud
WIDTH, HEIGHT = 700, 300

# get categories (sorted for consistent layout)
categories = sorted(df_clean['category'].dropna().unique().tolist())

# prepare subplots: 2 rows x 4 cols
fig, axes = plt.subplots(2, 4, figsize=(18, 8))
axes = axes.ravel()  # flatten to 1D index

# for loop through the list of categories
for i, cat in enumerate(categories):
    # set axis for category
    ax = axes[i]
    # aggregate tokens for content_tokens category
    tokens_series = df_clean.loc[df_clean['category'] == cat, 'comb_tokens']
    freq = Counter()
    for toks in tokens_series:
        if isinstance(toks, list):
            # light filter for readability
            freq.update([t for t in toks if isinstance(t, str) and len(t) >= MIN_TOKEN_LEN])
    # subplot formatting
    ax.set_title(str(cat), fontsize=12, pad=8)
    ax.axis("off")
    # handle the empty cases
    if len(freq) == 0:
        ax.text(0.5, 0.5, "No tokens", ha="center", va="center", fontsize=12)
        continue

    # create the wordcloud itself
    wc = WordCloud(
        width=WIDTH,
        height=HEIGHT,
        background_color="white",
        colormap="plasma",
        max_words=MAX_WORDS,
        normalize_plurals=True
    ).generate_from_frequencies(freq)

    ax.imshow(wc, interpolation="bilinear")

# hide any unused axes
for j in range(len(categories), 8):
    axes[j].axis("off")

fig.suptitle("Wordclouds of Content Tokens by Category", fontsize=20)
plt.show()

In [ ]:
# Build out wordclouds for the title_tokens of each category.
# Config
MAX_WORDS = 200
MIN_TOKEN_LEN = 2
# pixel size passed to WordCloud
WIDTH, HEIGHT = 700, 300

# get categories (sorted for consistent layout)
categories = sorted(df_clean['category'].dropna().unique().tolist())

# prepare subplots: 2 rows x 4 cols
fig, axes = plt.subplots(2, 4, figsize=(18, 8))
axes = axes.ravel()  # flatten to 1D index

# for loop through the list of categories
for i, cat in enumerate(categories):
    # set axis for category
    ax = axes[i]
    # aggregate tokens for content_tokens category
    tokens_series = df_clean.loc[df_clean['category'] == cat, 'title_tokens']
    freq = Counter()
    for toks in tokens_series:
        if isinstance(toks, list):
            # light filter for readability
            freq.update([t for t in toks if isinstance(t, str) and len(t) >= MIN_TOKEN_LEN])
    # subplot formatting
    ax.set_title(str(cat), fontsize=12, pad=8)
    ax.axis("off")
    # handle the empty cases
    if len(freq) == 0:
        ax.text(0.5, 0.5, "No tokens", ha="center", va="center", fontsize=12)
        continue

    # create the wordcloud itself
    wc = WordCloud(
        width=WIDTH,
        height=HEIGHT,
        background_color="white",
        colormap="plasma",
        max_words=MAX_WORDS,
        normalize_plurals=True
    ).generate_from_frequencies(freq)

    ax.imshow(wc, interpolation="bilinear")

# hide any unused axes
for j in range(len(categories), 8):
    axes[j].axis("off")

fig.suptitle("Wordclouds of Title Tokens by Category", fontsize=20)
plt.show()

In [ ]:
# Build out wordclouds for the desc_tokens of each category.
# Config
MAX_WORDS = 200
MIN_TOKEN_LEN = 2
# pixel size passed to WordCloud
WIDTH, HEIGHT = 700, 300

# get categories (sorted for consistent layout)
categories = sorted(df_clean['category'].dropna().unique().tolist())

# prepare subplots: 2 rows x 4 cols
fig, axes = plt.subplots(2, 4, figsize=(18, 8))
axes = axes.ravel()  # flatten to 1D index

# for loop through the list of categories
for i, cat in enumerate(categories):
    # set axis for category
    ax = axes[i]
    # aggregate tokens for content_tokens category
    tokens_series = df_clean.loc[df_clean['category'] == cat, 'desc_tokens']
    freq = Counter()
    for toks in tokens_series:
        if isinstance(toks, list):
            # light filter for readability
            freq.update([t for t in toks if isinstance(t, str) and len(t) >= MIN_TOKEN_LEN])
    # subplot formatting
    ax.set_title(str(cat), fontsize=12, pad=8)
    ax.axis("off")
    # handle the empty cases
    if len(freq) == 0:
        ax.text(0.5, 0.5, "No tokens", ha="center", va="center", fontsize=12)
        continue

    # create the wordcloud itself
    wc = WordCloud(
        width=WIDTH,
        height=HEIGHT,
        background_color="white",
        colormap="plasma",
        max_words=MAX_WORDS,
        normalize_plurals=True
    ).generate_from_frequencies(freq)

    ax.imshow(wc, interpolation="bilinear")

# hide any unused axes
for j in range(len(categories), 8):
    axes[j].axis("off")

fig.suptitle("Wordclouds of Descriptive Tokens by Category", fontsize=20)
plt.show()

In [ ]:
# use the descriptive stats for each _tokens column
# identify the columns to use
token_cols = ['title_tokens', 'desc_tokens', 'content_tokens']
# create an empty dictionary
desc_stats_summary = {}

# for loop
for col in token_cols:
    # flatten column tokens into list
    col_tokens = [t for tokens in df_clean[col] if isinstance(tokens, list) for t in tokens]
    # apply desc stats function
    stats = descriptive_stats(col_tokens, verbose=False)
    # add results to the dictionary
    desc_stats_summary[col] = {
        "Total Tokens": stats[0],
        "Unique Tokens": stats[1],
        "Lexical Diversity": round(stats[2], 3),
        "Total Characters": stats[3]
    }

# convert to df to view and transpose
desc_stats_df = pd.DataFrame(desc_stats_summary).T
display(desc_stats_df)

## Data Splitting and Class Imbalance

In [ ]:
from sklearn.model_selection import train_test_split

# subset only the required columns and drop rows with missing values
df_subset = df_clean[['category', 'comb_text']].dropna()

# Perform stratified split to maintain class proportions
train_df, test_df = train_test_split(
    df_subset, 
    test_size=0.2, 
    random_state=42, 
    stratify=df_subset['category']  # This ensures proportional split
)

# Verify the proportions were maintained
print("\nCategory distribution in training set:")
print(train_df['category'].value_counts(normalize=True).round(3))
print("\nCategory distribution in test set:")
print(test_df['category'].value_counts(normalize=True).round(3))

print(f"\nTrain shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

## Model Development (Training and Testing)

### Logistic Regression

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Create a pipeline: TF-IDF vectorizer + Logistic Regression
logreg_pipeline = make_pipeline(
    TfidfVectorizer(max_features=10000, ngram_range=(1,2)),
    LogisticRegression(max_iter=1000, random_state=42)
)

# Fit the model
logreg_pipeline.fit(train_df['comb_text'], train_df['category'])

# Predict on test set
y_pred = logreg_pipeline.predict(test_df['comb_text'])

# Calculate accuracy
accuracy = accuracy_score(test_df['category'], y_pred)

# Get classification report as dict
report = classification_report(test_df['category'], y_pred, output_dict=True)

# Convert to results DataFrame
classifier_results = pd.DataFrame(report).T
classifier_results['accuracy'] = accuracy  # Add accuracy column for all rows

# Display summary
display(classifier_results)

### SVM

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

# Create a pipeline: TF-IDF vectorizer + Linear SVM
svm_pipeline = make_pipeline(
    TfidfVectorizer(max_features=10000, ngram_range=(1,2)),
    LinearSVC(max_iter=1000, random_state=42)
)

# Fit the model
svm_pipeline.fit(train_df['comb_text'], train_df['category'])

# Predict on test set
y_pred_svm = svm_pipeline.predict(test_df['comb_text'])

# Calculate accuracy
svm_accuracy = accuracy_score(test_df['category'], y_pred_svm)

# Get classification report as dict
svm_report = classification_report(test_df['category'], y_pred_svm, output_dict=True)

# Convert to results DataFrame
svm_classifier_results = pd.DataFrame(svm_report).T
svm_classifier_results['accuracy'] = svm_accuracy  # Add accuracy column for all rows

# Display summary
display(svm_classifier_results)

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

# Create a pipeline: TF-IDF vectorizer + Multinomial Naive Bayes
nb_pipeline = make_pipeline(
    TfidfVectorizer(max_features=10000, ngram_range=(1,2)),
    MultinomialNB()
)

# Fit the model
nb_pipeline.fit(train_df['comb_text'], train_df['category'])

# Predict on test set
y_pred_nb = nb_pipeline.predict(test_df['comb_text'])

# Calculate accuracy
nb_accuracy = accuracy_score(test_df['category'], y_pred_nb)

# Get classification report as dict
nb_report = classification_report(test_df['category'], y_pred_nb, output_dict=True)

# Convert to results DataFrame
nb_classifier_results = pd.DataFrame(nb_report).T
nb_classifier_results['accuracy'] = nb_accuracy  # Add accuracy column for all rows

# Display summary
display(nb_classifier_results)

## Topic Modeling

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import silhouette_score
import numpy as np

# Vectorize the text
vectorizer = CountVectorizer(
    max_features=5000,
    stop_words='english'
)
X_train_counts = vectorizer.fit_transform(train_df['comb_text'])

# Fit LDA model
n_topics = 7  # You have 7 categories, so let's use 7 topics
lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=10,
    learning_method='online',
    random_state=42
)
lda.fit(X_train_counts)

# Display top words by topic
def print_top_words(model, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        top_features = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print(f"Topic #{topic_idx + 1}: {', '.join(top_features)}")

print_top_words(lda, vectorizer.get_feature_names_out())

# Assign each document to its most probable topic
topic_assignments = lda.transform(X_train_counts).argmax(axis=1)
train_df['lda_topic'] = topic_assignments

### EVALUATION

# Evaluate topic coherence with silhouette score
try:
    sil_score = silhouette_score(lda.transform(X_train_counts), topic_assignments)
    print(f"\nSilhouette Score (topic separation): {sil_score:.3f}")
except Exception as e:
    print(f"\nSilhouette Score could not be computed: {e}")

# Show topic distribution counts
topic_counts = train_df['lda_topic'].value_counts().sort_index()
print("\nDocument count per topic:")
print(topic_counts)

# For each topic, find the most common category in train_df
topic_to_category = (
    train_df.groupby('lda_topic')['category']
    .agg(lambda x: x.value_counts().idxmax())
    .rename('most_common_category')
)

# Show the mapping of topic to most common category
print(topic_to_category)

## Results